In [ ]:
# Import modules

import sys
import pandas as pd
import numpy as np
import random
import pickle
import datetime
import matplotlib.pyplot as plt
%matplotlib inline
from statsmodels.tsa.seasonal import seasonal_decompose

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

In [ ]:
df = pd.read_pickle('X:/Brondata/donatiedata_200601_201902.pkl')
df_donors = pd.read_pickle('X:/Brondata/donors.pkl')

In [ ]:
# List of KeyIDs by sex
id_f = list(df_donors.loc[df_donors['Geslacht'] == 'V', ].index)
id_m = list(df_donors.loc[df_donors['Geslacht'] == 'M', ].index)

In [ ]:
df['Date'] = df['DateTime'].dt.date
df['Time'] = df['DateTime'].dt.time
df.head()

In [ ]:
plt.plot(df.loc[df['KeyID'].isin(id_f), ].groupby(['DateTime'])['Hb'].mean())

In [ ]:
plt.plot(df.loc[df['KeyID'].isin(id_m), ].groupby(['DateTime'])['Hb'].mean())

In [ ]:
seriesF = df.loc[df['KeyID'].isin(id_f), ].groupby(['DateTime'])['Hb'].mean().to_frame()
seriesF = seriesF.dropna(subset=['Hb'])
seriesF = seriesF.reset_index()
seriesF['DateTime'] = pd.to_datetime(seriesF['DateTime'])
seriesF = seriesF.sort_values('DateTime').set_index('DateTime')
seriesF = seriesF.resample('W-Mon').mean()

resultF = seasonal_decompose(seriesF, model='additive', freq=52)
resultF.plot()
plt.show()

In [ ]:
seriesM = df.loc[df['KeyID'].isin(id_m), ].groupby(['DateTime'])['Hb'].mean().to_frame()
seriesM = seriesM.dropna(subset=['Hb'])
seriesM = seriesM.reset_index()
seriesM['DateTime'] = pd.to_datetime(seriesM['DateTime'])
seriesM = seriesM.sort_values('DateTime').set_index('DateTime')
seriesM = seriesM.resample('W-Mon').mean()

resultM = seasonal_decompose(seriesM, model='additive', freq=52)
resultM.plot()
plt.show()

In [ ]:
def adjustHb(row, adjustment):
    lastmonday = pd.to_datetime(row['DateTime']) - datetime.timedelta(days=row['DateTime'].weekday())
    return row['Hb'] - float(adjustment.loc[lastmonday, ])

In [ ]:
# 16:10 - 16:36
df.loc[df['KeyID'].isin(id_f), 'Hb_adjusted'] = df.loc[df['KeyID'].isin(id_f), ].apply(adjustHb, adjustment=resultF.seasonal, axis=1)

In [ ]:
# 13:44
df.loc[df['KeyID'].isin(id_m), 'Hb_adjusted'] = df.loc[df['KeyID'].isin(id_m), ].apply(adjustHb, adjustment=resultM.seasonal, axis=1)

In [ ]:
df.to_pickle('X:/201902 Hb Trajectories/donaties_hb_adjusted.pkl')

In [ ]:
df = pd.read_pickle('X:/201902 Hb Trajectories/donaties_hb_adjusted.pkl')
df.head()

In [ ]:
seriesF = df.loc[df['KeyID'].isin(id_m), ].groupby(['DateTime'])['Hb_adjusted'].mean().to_frame()
seriesF = seriesF.dropna(subset=['Hb_adjusted'])
seriesF = seriesF.reset_index()
seriesF['DateTime'] = pd.to_datetime(seriesF['DateTime'])
seriesF = seriesF.sort_values('DateTime').set_index('DateTime')
seriesF = seriesF.resample('W-Mon').mean()

resultF = seasonal_decompose(seriesF, model='additive', freq=52)
resultF.plot()
plt.show()